In [ ]:
import pandas as pd
import os
from pathlib import Path

# Step 1: Ask for folder path & filenames
folder_path = input("📁 Enter folder path containing all 4 Excel files: ").strip()

# Get each filename from user
gsx_file = input("🔹 Enter GSX file name (e.g. gsx.xlsx): ").strip()
gsx_bill_file = input("🔹 Enter GSX Bill file name: ").strip()
uni_bill_file = input("🔹 Enter UNI Bill file name: ").strip()
crm_file = input("🔹 Enter CRM file name: ").strip()

# Combine with folder path
path_gsx = os.path.join(folder_path, gsx_file)
path_gsxbill = os.path.join(folder_path, gsx_bill_file)
path_unibill = os.path.join(folder_path, uni_bill_file)
path_crm = os.path.join(folder_path, crm_file)

# Step 2: Read all files
gsx = pd.read_excel(path_gsx, dtype=str)
gsx_bill = pd.read_excel(path_gsxbill, dtype=str)
uni_bill = pd.read_excel(path_unibill, dtype=str)
crm = pd.read_excel(path_crm, dtype=str)

# Step 3: Filter GSX for parts containing "ACSH"
gsx_filtered = gsx[gsx['Part Number'].str.contains('ACSH', case=False, na=False)].copy()

# Step 4: Rename columns with suffixes
gsx_cols = [
    'Repair', 'Serial Number', 'Repair Type', 'Product Name', 'Purchase Order',
    'Coverage Status', 'Reference', 'Service Provider Ship-To',
    'Customer Reported Component Issue', 'Technician Verified Component Issue',
    'Repair Classification', 'Part Number', 'Invoice Created'
]
gsx_filtered = gsx_filtered[gsx_cols]
gsx_filtered.columns = [col + "_gsx" for col in gsx_filtered.columns]

crm_cols = [
    'reference_number', 'serial_number', 'product_name', 'coverage_type',
    'outward_condition', 'voucher_amount', 'gsx_reference_number',
    'gsx_repair_type', 'gsx_service_type', 'part_number'
]
crm = crm[crm_cols]
crm.columns = [col + "_crm" for col in crm.columns]

gsx_bill_cols = ['Total Amount', 'Repair', 'Invoice', 'Invoice Type']
gsx_bill = gsx_bill[gsx_bill_cols]
gsx_bill.columns = [col + "_gsxbill" for col in gsx_bill.columns]

uni_bill_cols = ['Ref. No', 'Bill amount', 'Remarks']
uni_bill = uni_bill[uni_bill_cols]
uni_bill.columns = [col + "_unibill" for col in uni_bill.columns]

# Step 5: Merge steps
m1 = pd.merge(
    gsx_filtered, crm,
    left_on='Repair_gsx', right_on='gsx_reference_number_crm',
    how='left'
)

m2 = pd.merge(
    m1, gsx_bill,
    left_on='Purchase Order_gsx', right_on='Repair_gsxbill',
    how='left'
)

m3 = pd.merge(
    m2, uni_bill,
    left_on='Purchase Order_gsx', right_on='Ref. No_unibill',
    how='left'
)

# Step 6: Find mismatches
mismatches = m3[
    (m3['voucher_amount_crm'] != m3['Total Amount_gsxbill']) |
    (m3['voucher_amount_crm'] != m3['Bill amount_unibill']) |
    (m3['Total Amount_gsxbill'] != m3['Bill amount_unibill'])
].copy()

# Step 7: Output to Desktop
desktop_path = str(Path.home() / "Desktop")
output_path = os.path.join(desktop_path, "AC+amount_mismatch.xlsx")

mismatches.to_excel(output_path, index=False)
print(f"✅ Mismatches saved to: {output_path}")
